In [105]:
import tensorflow as tf
from tensorflow import keras
import torch
torch.cuda.is_available()

False

In [97]:
tf.keras.datasets.mnist.load_data(path="mnist.npz")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [98]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [99]:
x_train = x_train.reshape((60000,1, 28,28))
x_test = x_test.reshape((10000, 1 , 28, 28))
y_train = y_train.reshape((60000, 1,1,1))
y_test = y_test.reshape((10000, 1,1,1))

In [100]:
x_train = torch.from_numpy(x_train)
x_test = torch.from_numpy(x_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [101]:
x_train.device

device(type='cpu')

In [104]:
torch.cuda.is_available()

False

In [103]:
x_train = x_train.to(torch.device('cuda'))
x_test = x_test.to(torch.device('cuda'))
y_train = y_train.to(torch.device('cuda'))
y_test = y_test.to(torch.device('cuda'))



AssertionError: Torch not compiled with CUDA enabled

<h2>가중치,편향등 변수 초기화

In [ ]:
import numpy as np

#test_x = np.ones((1,1,28,28))

kernel1 = np.ones((5,10,10))
kernel2 = np.ones((10,5,5))
w1 = np.ones((2250,160))
w2 = np.ones((160,10))

b1 = np.ones((5))
b2 = np.ones((10))
b3 = np.ones((160))
b4 = np.ones((10))

<h2>함수 정의

<h5>탄젠트 함수

In [ ]:
def tahn(x):
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

<h5>컨볼루션 연산 함수

In [ ]:
def conv2D(input,kernel,strides):
    datasize,x_channel,x_width,x_height = input.shape

    kernels,kernel_height,kernel_width = kernel.shape
    
    result_height = (x_height - kernel_height) // strides + 1
    result_width = (x_width - kernel_width) // strides + 1

    result = np.zeros((datasize, kernels, result_height, result_width))

    for i in range(datasize):
        for j in range(kernels):
            for k in range(x_channel):
                for l in range(result_height):
                    for n in range(result_width):
                        h_start = l * strides
                        h_end = h_start + kernel_height
                        w_start = n * strides
                        w_end = w_start + kernel_width
                        result[i, j, l, n] += np.sum(input[i, k, h_start:h_end, w_start:w_end] * kernel[j, k])
    return result
                                                                        

<h5> 밀집층 함수

In [ ]:
def dense(input, weight, bias):
    output_size = len(weight[1])
    output = np.zeros((output_size))
    output = np.dot(input, weight)

    for i in range(output_size):
        output[i] = tahn(output[i])
    output += bias

    return output


[8 9]


In [ ]:
result = conv2D(x_test,kernel1,1)
print(result.shape)
result = conv2D(result,kernel2,2)
print(result.shape)
result = result.flatten()
print(result.shape)

# result = dense(result,w1,b3)
# print(result.size)



(10000, 5, 19, 19)
(10000, 10, 8, 8)
(6400000,)


<h1>모델 구성

In [ ]:
model = tf.keras.Sequential([ 
tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
tf.keras.layers.Conv2D(3,7,strides=4,activation="relu",padding="same", input_shape=[28,28,1]),
tf.keras.layers.MaxPooling2D(2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(16,activation="relu"),
tf.keras.layers.Dense(10,activation="softmax")
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train)


1875/1875 [==============================] - 2s 938us/step - loss: 0.8918 - accuracy: 0.7154


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train)


1875/1875 [==============================] - 10s 5ms/step - loss: 0.2353 - accuracy: 0.9295


In [ ]:
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
y_train = y_train.reshape((60000, 1,1,1))
y_test = y_test.reshape((10000, 1,1,1))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
    tf.keras.layers.ZeroPadding2D(padding=2),
    tf.keras.layers.Conv2D(filters=6,kernel_size=5,strides=1,activation="tanh",padding="valid"),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2),
    tf.keras.layers.Conv2D(filters=16,kernel_size=5,strides=1,activation="tanh",padding="valid"),                       
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2),strides=2),    
    tf.keras.layers.Conv2D(filters=120,kernel_size=5,strides=1,activation="tanh",padding="valid"),
    tf.keras.layers.Dense(84,activation="tanh"),
    tf.keras.layers.Dense(10,activation="softmax")
    ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy']),
model.fit(x_train, y_train)

1875/1875 [==============================] - 8s 4ms/step - loss: 0.2281 - accuracy: 0.9311


In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print("테스트 정확도 : %d" %(test_acc*100))

313/313 - 1s - loss: 0.0945 - accuracy: 0.9718 - 687ms/epoch - 2ms/step
테스트 정확도 : 97


In [ ]:
print(y.shape,y_train.shape)

(60000, 1, 1, 10) (60000, 1, 1, 1)


In [ ]:
print(y[0])

[[[2.2964303e-05 2.2208309e-04 6.3666203e-06 3.4293845e-02 7.2778594e-06
   9.6467173e-01 2.0611154e-05 6.6860630e-05 3.3428002e-04 3.5394510e-04]]]


In [ ]:
y=model.predict(x_train)
print(y.shape,x_train.shape)

1875/1875 [==============================] - 4s 2ms/step
(60000, 1, 1, 10) (60000, 28, 28, 1)


In [ ]:
print(y[0])

[[[5.8902242e-05 3.2386612e-05 9.3773393e-05 2.9462626e-02 6.8596560e-06
   9.6922696e-01 6.5715773e-05 3.9365950e-05 6.1171700e-04 4.0169200e-04]]]
